In [1]:
import string # for lowercase letter list
import numpy as np
import math

In [2]:
with open('/usr/share/dict/words') as o:
    DICTIONARY = set([s.strip() for s in o.readlines() if not s[0].isupper()])

In [3]:
LETTERS =  string.ascii_lowercase

In [128]:
class RandomHangManStrategy:
 
    def start(self):
        """Reset any internal state"""
        np.random.seed(1)
        
    def guess(self, word, excluded_letters, possibilities):
        possible_letter = [letter for letter in LETTERS if not letter in excluded_letters]
        return np.random.choice(possible_letter, 1)[0]

In [143]:
class MaxEntropyHangManStrategy:
    """Guess picks the letter that maximises the product E * I where E and I are 
    the number of words excluded (E) or included (I) by a given guess."""
    def start(self):
        """Reset any internal state"""
        pass
    
    def guess(self, word, excluded_letters, possibilities):
        possible_letters = [letter for letter in LETTERS if not letter in excluded_letters]
        
        include_exclude_counts = {}
        for letter in possible_letters:
            include_count = 0
            for word in possibilities:
                if letter in word:
                    include_count += 1
            include_exclude_counts[letter] = (include_count, len(possibilities) - include_count)
        
        best_letter, max_measure = '?', -1
        for letter, (include_count, exlude_count) in include_exclude_counts.items():
            measure = include_count * exclude_count
            if measure > max_measure and letter not in excluded_letters:
                max_measure = measure
                best_letter = letter
                
        return best_letter

In [144]:
class SimpleHangManStrategy:
    """Guess picks the most common letter in the remaining possible words."""
    def start(self):
        """Reset any internal state"""
        pass
    
    def guess(self, word, excluded_letters, possibilities):
        letter_counts = {}
        for word in possibilities:
            for letter in word:
                if letter in letter_counts:
                    letter_counts[letter] += 1
                else:
                    letter_counts[letter] = 1
        
        most_common_letter, max_count = '?', 0
        for letter, count in letter_counts.items():
            if count > max_count and letter not in excluded_letters:
                max_count = count
                most_common_letter = letter
                
        return most_common_letter

In [145]:
class VowelHangManStrategy:
    """Guess picks the most common letter in the remaining possible words but weights 
    some letters according to a supplied bias.
    
    When `bias == 1` this strategy is the same as the SimpleHangManStrategy"""
    
    def __init__(self, bias=1):
        self.common_letters = ['a', 'e', 'i', 'o', 'u', 'y']
        self.bias = bias
    
    def start(self):
        """Reset any internal state"""
        pass
    
    def guess(self, word, excluded_letters, possibilities):
        letter_counts = {}
        for word in possibilities:
            for letter in word:
                if letter in self.common_letters:
                    if letter in letter_counts:
                        letter_counts[letter] += 1 * self.bias
                    else:
                        letter_counts[letter] = 1 * self.bias
                else:
                    if letter in letter_counts:
                        letter_counts[letter] += 1
                    else:
                        letter_counts[letter] = 1
        
        most_common_letter, max_count = '?', 0
        for letter, count in letter_counts.items():
            if count > max_count and letter not in excluded_letters:
                max_count = count
                most_common_letter = letter
                
        return most_common_letter

In [146]:
class HangManPlayer:
    
    def start(self, word_length):
        self.excluded_letters = []
        self.word = '?' * word_length
        self.possibilities = sorted([word for word in DICTIONARY if len(word) == word_length])
        self.guesses = 0
        self._strategy.start()
    
    def __init__(self, strategy):
        self.name = type(strategy).__name__
        self._strategy = strategy
        
    def guess(self):
        if '?' not in self.word:
            return f'The word is {self.word}. Guessed in {self.guesses} tries'
        
        if len(self.possibilities) == 0:
            return f'No possible words to try: {self.word}' 
        
        self.guesses += 1
        return self._strategy.guess(self.word, self.excluded_letters, self.possibilities)
    
    def finished(self):
        return '?' not in self.word
    
    @staticmethod
    def _keep_word(word, letter, positions):
        for i in range(len(word)):
            if word[i] != letter and i in positions:
                return False
            elif word[i] == letter and i not in positions:
                return False
        return True
    
    def update_word(self, letter, positions=None):
        positions = positions or []
        self.excluded_letters.append(letter)
 
        # Update word
        updated_word = [letter for letter in self.word]
        for position in positions:
            if updated_word[position] != '?':
                raise Exception(f'Letter at position {position} is already specified as {updated_word[position]}')
            updated_word[position] = letter
        self.word = ''.join(updated_word)

        # Update possibilities
        self.possibilities = [
            word for word in self.possibilities 
            if HangManPlayer._keep_word(word, letter, positions)
        ]

In [147]:
class HangManGame:
    
    def __init__(self, word, player):
        self.word = word
        self.player = player
        self.steps = 0
        
    def finished(self):
        return self.player.finished()
        
    def step(self):
        assert not self.finished(), "Game is finished!"
        self.steps += 1
        guess = self.player.guess()
        positions = []
        for i, letter in enumerate(self.word):
            if letter == guess:
                positions.append(i)
        self.player.update_word(guess, positions)

In [148]:
def run_game(word, player):
    if word not in DICTIONARY:
        raise Exception(f"Suggested word '{word}' is not in the provided dictionary - is the spelling correct?")
    game = HangManGame(word, player)
    player.start(len(word))

    while not game.finished():
        game.step()
    print(f"{player.name} got '{word}' in {game.steps}.")

In [149]:
def compare_players(word, players):
    for player in players:
        run_game(word, player) 

In [150]:
players = [
    HangManPlayer(SimpleHangManStrategy()),
    HangManPlayer(VowelHangManStrategy(bias=4)),
    HangManPlayer(RandomHangManStrategy()),
    HangManPlayer(MaxEntropyHangManStrategy())
]
np.random.seed(101)
for word in np.random.choice([w for w in DICTIONARY], 50):
    compare_players(word, players)
    print('\n' + '='*80 + '\n')

SimpleHangManStrategy got 'shoutingly' in 12.
VowelHangManStrategy got 'shoutingly' in 13.
RandomHangManStrategy got 'shoutingly' in 26.


NameError: name 'exclude_count' is not defined